In [1]:
%cd ../

In [2]:
RANDOM_SEED = 42
SUBSET_SAMPLE_DATA_PERCENTAGE = 0.1
MIN_IOU_REQUIRED = 0.7  # The minimum IOU required to be a good detection (ie higher is more strict)
NUM_PROPOSALS = 300
CHECKPOINT_DIR = './idd_cars_drop_checkpoints_with_nms'

%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
import torch as t
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
import numpy as np
import pickle
import h5py
from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt

from data.util import  read_image
from utils.vis_tool import vis_bbox
from utils import array_tool as at
from utils.config import opt
from utils.idd_utils import get_annotations
from model.utils.bbox_tools import bbox_iou

from data.dataset import Dataset, TestDataset, KittiDataset
from torch.utils.data import DataLoader
from utils.vis_tool import vis_bbox, vis_image
from utils.idd_utils import IndiaDrivingDataset
from ood_metrics import auroc, plot_roc, calc_metrics
from sklearn import svm
import pickle
import itertools
import torch
from model.faster_rcnn import nograd
from data.dataset import preprocess
from utils.mahalanobis import Mahal_Dist


import warnings
warnings.filterwarnings(action='once')

np.random.seed(RANDOM_SEED)

In [3]:
opt.voc_data_dir = "/home/tadenoud/Documents/kitti/VOC2012/"

kitti_val_dataset = KittiDataset(opt, split='val')
kitti_val_samples = len(kitti_val_dataset)

In [4]:
faster_rcnn = FasterRCNNVGG16()
trainer = FasterRCNNTrainer(faster_rcnn, visdom=False).cuda()

In [5]:
%ls ./checkpoints/cars_drop/

fasterrcnn_11062316_0.7550167879591324


In [6]:
save_dir = './checkpoints/cars_drop'
trainer.load(os.path.join(save_dir, 'fasterrcnn_11062316_0.7550167879591324'))
trainer.faster_rcnn.eval()

FasterRCNNVGG16(
  (extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=

In [7]:
model = trainer.faster_rcnn

In [8]:
# Get a single image
dataloader = DataLoader(kitti_val_dataset,
                batch_size=1,
                shuffle=False,
                pin_memory=True,
                num_workers=1)

imgs, sizes, gt_bboxes, gt_labels, _ = next(iter(dataloader))
sizes = [sizes[0][0].item(), sizes[1][0].item()]
gt_bboxes = gt_bboxes.squeeze(dim=0)
gt_labels = gt_labels.squeeze(dim=0)


#     # Get predictions and all classifier features
#     res = model.predict_with_features(imgs, visualize=True, perturbation=perturbation, temperature=temperature)
#     pred_bboxes, pred_labels, pred_scores, pred_features = res
#     pred_bboxes, pred_labels, pred_scores, pred_features = \
#             pred_bboxes[0], pred_labels[0], pred_scores[0], pred_features[0]

#     ious = bbox_iou(pred_bboxes, np.concatenate((gt_bboxes, np.zeros((1,4)))))
#     max_ious = np.nan_to_num(ious.max(axis=1))

#     max_aligned_gt_label = ious.argmax(axis=1)
#     true_label_per_pred = np.array([gt_labels[j] for j in max_aligned_gt_label])
#     pred_label_correct = true_label_per_pred == pred_labels

In [9]:
def max_softmax():
    res = model.predict_with_features(imgs, visualize=True)
    return res

In [10]:
%%timeit -r 10
max_softmax()

/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:638: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)


328 ms ± 17.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [11]:
def odin():
    res = model.predict_with_features(imgs, visualize=True, temperature=1000, perturbation=0.002)
    return res

In [12]:
%%timeit -r 10
odin()

/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:664: UserWarning: Legacy autograd function object was called twice.  You will probably get incorrect gradients from this computation, as the saved tensors from the second invocation will clobber the saved tensors from the first invocation.  Please consider rewriting your autograd function in the modern style; for information on the new format, please see: https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd


1.13 s ± 66.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [13]:
mahal_model_name = os.path.join(CHECKPOINT_DIR, "kitti-cars-fc-4096-4-mahalanobis.h5")
md = Mahal_Dist.load(mahal_model_name)

def mahal():
    res = model.predict_with_features(imgs, visualize=True)
    pred_bboxes, pred_labels, pred_scores, pred_features = res
    pred_bboxes, pred_labels, pred_scores, pred_features = \
            pred_bboxes[0], pred_labels[0], pred_scores[0], pred_features[0]
    return md(pred_features[4])

In [14]:
%%timeit -r 10
mahal()

352 ms ± 17.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [15]:
# Train a one-class SVM networks with those features
ONE_CLASS_SAVE_PATH = os.path.join(CHECKPOINT_DIR, "kitti-fc-4096-1-one-class.pickle")    
with open(ONE_CLASS_SAVE_PATH, 'rb') as f:
    clf = pickle.load(f)

def svm_fc():
    res = model.predict_with_features(imgs, visualize=True)
    pred_bboxes, pred_labels, pred_scores, pred_features = res
    pred_bboxes, pred_labels, pred_scores, pred_features = \
            pred_bboxes[0], pred_labels[0], pred_scores[0], pred_features[0]
    
    return clf.predict(pred_features[1])

/home/tadenoud/anaconda3/envs/nn/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator OneClassSVM from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [16]:
%%timeit -r 10
svm_fc()

/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:638: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:638: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:638: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch

809 ms ± 237 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [17]:
ONE_CLASS_SAVE_PATH = os.path.join(CHECKPOINT_DIR, "kitti-conv-512-one-class.pickle")
with open(ONE_CLASS_SAVE_PATH, 'rb') as f:
    clf = pickle.load(f)

def svm_conv():
    res = model.predict_with_features(imgs, visualize=True)
    pred_bboxes, pred_labels, pred_scores, pred_features = res
    pred_bboxes, pred_labels, pred_scores, pred_features = \
            pred_bboxes[0], pred_labels[0], pred_scores[0], pred_features[0]
    
    return clf.predict(pred_features[0])

/home/tadenoud/anaconda3/envs/nn/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator OneClassSVM from version 0.21.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [18]:
%%timeit -r 10
svm_conv()

/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:638: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:638: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:638: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch

944 ms ± 56.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [19]:
def svm_vahdat():
    pass

In [20]:
%%timeit -r 10
svm_vahdat()

95.2 ns ± 14.3 ns per loop (mean ± std. dev. of 10 runs, 10000000 loops each)


In [21]:
import mc_uncertainty as mcu
from model.utils.bbox_tools import loc2bbox, bbox2loc, bbox_iou
from torch.nn import functional as F

def _predict_mc(self, imgs, sizes=None, visualize=False, num_passes=1):
    if visualize:
        self.use_preset('visualize')
        prepared_imgs = list()
        sizes = list()
        for img in imgs:
            size = img.shape[1:]
            img = preprocess(at.tonumpy(img))
            prepared_imgs.append(img)
            sizes.append(size)
    else:
         prepared_imgs = imgs 
    bboxes = list()
    labels = list()
    scores = list()
    mc_scores = list()

    for img, size in zip(prepared_imgs, sizes):
        scale = img.shape[2] / size[1]

        img = at.totensor(img[None], cuda=True).float()

        h = self.extractor(img)
        rpn_locs, rpn_scores, rois, roi_indices, anchor = self.rpn(h, img.shape[2:], scale)
        roi_cls_locs, roi_scores = self.head(h, rois, roi_indices)
        
        roi_scores = [roi_scores] + [self.head(h, rois, roi_indices)[1] for _ in range(num_passes-1)]

        # We are assuming that batch size is 1.
        roi_score = [s.data for s in roi_scores]
        roi_cls_loc = roi_cls_locs.data
        roi = at.totensor(rois) / scale

        # Convert predictions to bounding boxes in image coordinates.
        # Bounding boxes are scaled to the scale of the input images.
        mean = t.Tensor(self.loc_normalize_mean).cuda(). \
            repeat(self.n_class)[None]
        std = t.Tensor(self.loc_normalize_std).cuda(). \
            repeat(self.n_class)[None]

        roi_cls_loc = (roi_cls_loc * std + mean)
        roi_cls_loc = roi_cls_loc.view(-1, self.n_class, 4)
        roi = roi.view(-1, 1, 4).expand_as(roi_cls_loc)
        cls_bbox = loc2bbox(at.tonumpy(roi).reshape((-1, 4)),
                            at.tonumpy(roi_cls_loc).reshape((-1, 4)))
        cls_bbox = at.totensor(cls_bbox)
        cls_bbox = cls_bbox.view(-1, self.n_class * 4)
        # clip bounding box
        cls_bbox[:, 0::2] = (cls_bbox[:, 0::2]).clamp(min=0, max=size[0])
        cls_bbox[:, 1::2] = (cls_bbox[:, 1::2]).clamp(min=0, max=size[1])

        prob = [at.tonumpy(F.softmax(at.totensor(s), dim=1)) for s in roi_score]

        raw_cls_bbox = at.tonumpy(cls_bbox)
        raw_prob = [at.tonumpy(p) for p in prob]

        bbox, label, score, mc_score = self._suppress_with_features(raw_cls_bbox, raw_prob[0], raw_prob)

        bboxes.append(bbox)
        labels.append(label)
        scores.append(score)
        mc_scores.append(mc_score)

    # self.use_preset('evaluate')

    return bboxes, labels, scores, mc_scores

model.predict_mc = _predict_mc.__get__(model)

In [22]:
def entropy():
    model.train()
    
    res = model.predict_mc(imgs, visualize=True, num_passes=10)
    pred_bboxes, pred_labels, pred_scores, pred_mc_scores = res
    pred_bboxes, pred_labels, pred_scores, pred_mc_scores = \
            pred_bboxes[0], pred_labels[0], pred_scores[0], np.array(pred_mc_scores[0])
    
    model.eval()
    
    return -mcu.predicted_entropy(pred_mc_scores)

In [23]:
%%timeit -r 10
entropy()

/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:638: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
/opt/conda/conda-bld/pytorch_1565272271120/work/torch/csrc/autograd/python_function.cpp:664: UserWarning: Legacy autograd function object was called twice.  You will probably get incorrect gradients from this computation, as the saved tensors from the second invocation will clobber the saved tensors from the first invocation.  Please consider rewriting your autograd function in the modern style; for information on the new format, please see: https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd


1.25 s ± 94.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [24]:
def mutual_information():
    model.train()
    
    res = model.predict_mc(imgs, visualize=True, num_passes=10)
    pred_bboxes, pred_labels, pred_scores, pred_mc_scores = res
    pred_bboxes, pred_labels, pred_scores, pred_mc_scores = \
            pred_bboxes[0], pred_labels[0], pred_scores[0], np.array(pred_mc_scores[0])

    model.eval()
    
    return mcu.mutual_information(pred_mc_scores)

In [25]:
%%timeit -r 10
mutual_information()

1.4 s ± 150 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [26]:
mahal_model_name = os.path.join(CHECKPOINT_DIR, "kitti-cars-fc-4096-4-mahalanobis.h5")
md = Mahal_Dist.load(mahal_model_name)

def mahal_pert():
    res = model.predict_with_features(imgs, visualize=True, temperature=1000, perturbation=0.002)
    pred_bboxes, pred_labels, pred_scores, pred_features = res
    pred_bboxes, pred_labels, pred_scores, pred_features = \
            pred_bboxes[0], pred_labels[0], pred_scores[0], pred_features[0]
    return md(pred_features[4])

In [27]:
%%timeit -r 10
mahal_pert()

1.36 s ± 130 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
